In [1]:
import pyscf
from pyscf import tools, symm
import numpy as np

In [77]:

molecule = """
N   0.0 0.0 0.0
N   2.0 0.0 0.0
"""


cas_norb = 34
cas_nel =34
focc = 93

basis = "cc-pvtz"
#

mol = pyscf.gto.Mole(atom=molecule,
    symmetry = True,
    spin   = 0,
    charge = 0,
    basis = basis)

# mol.symmetry_subgroup = "C2"
# mol.symmetry = 'c2h'
mol.build()

print("symmetry: ",mol.topgroup)
mf = pyscf.scf.RHF(mol)
mf.verbose = 4
mf.conv_tol = 1e-8
mf.conv_tol_grad = 1e-5
mf.init_guess = "sad"
mf.run(max_cycle=200)

print(" Hartree-Fock Energy: %12.8f" % mf.e_tot)


symmetry:  Dooh


******** <class 'pyscf.scf.hf_symm.SymAdaptedRHF'> ********
method = SymAdaptedRHF-RHF
initial guess = sad
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
SCF conv_tol = 1e-08
SCF conv_tol_grad = 1e-05
SCF max_cycles = 200
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = /var/folders/9p/nhnq58pj3td7svw_pppk_syw0000gn/T/tmpgkf7z608
max_memory 4000 MB (current use 0 MB)
init E= -107.67673318338
HOMO (E1uy) = -0.213036145190876  LUMO (E1gx) = -0.118212329290876
cycle= 1 E= -108.328975817741  delta_E= -0.652  |g|= 0.348  |ddm|=  1.2
HOMO (E1ux) = -0.458674672154445  LUMO (E1gy) = -0.147548474942933
cycle= 2 E= -108.354912280813  delta_E= -0.0259  |g|= 0.103  |ddm|= 0.305
HOMO (E1ux) = -0.409874221051098  LUMO (E1gy) = -0.0940299468241889
cycle= 3 E= -108.357302480392  delta_E= -0.00239  |g|= 0.0196  |ddm|= 0.0552
HOMO (E1ux) = -0.409684421365854  LUMO (E1gy) = -0.0926640103211947
cycl

In [78]:

norb = mf.mo_coeff.shape[0]
pyscf.tools.molden.from_mo(mol, "C_RHF.molden", mf.mo_coeff)
for s,i,c in zip(mol.irrep_name, mol.irrep_id, mol.symm_orb):
    print(" %5s %3i %4i"%(s, i, c.shape[1]))

def myocc(mf):    
    mol = mf.mol
    irrep_id = mol.irrep_id
    so = mol.symm_orb
    orbsym = symm.label_orb_symm(mol, irrep_id, so, mf.mo_coeff)
    occsym = np.array(orbsym)[mf.mo_occ==2]
    virsym = np.array(orbsym)[mf.mo_occ==0]
    for ir,irname in enumerate(mol.irrep_name):
        print('%5s, Occ = %5d, Vir = %d' % (irname, sum(occsym==ir), sum(virsym==ir)))
    print(orbsym)
    print(mol.irrep_id)
    print(mol.irrep_name)

myocc(mf)


   A1g   0   10
  E1gx   2    6
  E1gy   3    6
   A1u   5   10
  E1uy   6    6
  E1ux   7    6
  E2gx  10    3
  E2gy  11    3
  E3gx  12    1
  E3gy  13    1
  E2uy  14    3
  E2ux  15    3
  E3uy  16    1
  E3ux  17    1
  A1g, Occ =     3, Vir = 7
 E1gx, Occ =     0, Vir = 0
 E1gy, Occ =     0, Vir = 6
  A1u, Occ =     0, Vir = 6
 E1uy, Occ =     0, Vir = 0
 E1ux, Occ =     2, Vir = 8
 E2gx, Occ =     1, Vir = 5
 E2gy, Occ =     1, Vir = 5
 E3gx, Occ =     0, Vir = 0
 E3gy, Occ =     0, Vir = 0
 E2uy, Occ =     0, Vir = 3
 E2ux, Occ =     0, Vir = 3
 E3uy, Occ =     0, Vir = 1
 E3ux, Occ =     0, Vir = 1
[ 0  5  0  5  0  6  7  2  3  5  5  6  7  0  2  3  0  5  6  7 10 11  0 14
 15  2  3  5  0  6  7  2  3  5  6  7  0 10 11 16 17 12 13 14 15  2  3  5
  6  7 10 11 14 15  0  2  3  5  0  5]
[0, 2, 3, 5, 6, 7, 10, 11, 12, 13, 14, 15, 16, 17]
['A1g', 'E1gx', 'E1gy', 'A1u', 'E1uy', 'E1ux', 'E2gx', 'E2gy', 'E3gx', 'E3gy', 'E2uy', 'E2ux', 'E3uy', 'E3ux']


In [104]:
sym_map = {}
for ir,irname in enumerate(mol.irrep_id):
    sym_map[irname] = ir

clusters = []
for i in range(8):
    clusters.append([])
    
orb_syms = {}
for i in range(norb):
    # print(" Orb: %5i %5s %5s " %(i, mf.orbsym[i], mf.orbsym[i]))
    print(" Orb: %5i %5s " %(i,  mol.irrep_name[sym_map[mf.orbsym[i]]]))
    # orb_syms.append(mol.irrep_name[sym_map[mf.orbsym[i]]])
    name = mol.irrep_name[sym_map[mf.orbsym[i]]]
    if i not in frozen:
        if name == "A1g" or name == "A1u":
            clusters[1].append(i)
        elif name == "E1gx" or name == "E1ux":
            clusters[2].append(i)
        elif name == "E1gy" or name == "E1uy":
            clusters[3].append(i)
        elif name == "E2gx" or name == "E2ux":
            clusters[4].append(i)
        elif name == "E2gy" or name == "E2uy":
            clusters[5].append(i)
        elif name == "E3gx" or name == "E3ux":
            clusters[6].append(i)
        elif name == "E3gy" or name == "E3uy":
            clusters[7].append(i)
        
    if name in orb_syms:
        orb_syms[name].append(i)
    else:
        orb_syms[name] = [i]    
            
#specify frozen core
clusters[0] = [0,1]

C_ordered = np.zeros((norb,0))
for ci,c in enumerate(clusters):
    C_ordered = np.hstack((C_ordered, C[:,c]))
    

pyscf.tools.molden.from_mo(mol, "C_ordered.molden", C_new)


 Orb:     0   A1g 
 Orb:     1   A1u 
 Orb:     2   A1g 
 Orb:     3   A1u 
 Orb:     4   A1g 
 Orb:     5  E1uy 
 Orb:     6  E1ux 
 Orb:     7  E1gx 
 Orb:     8  E1gy 
 Orb:     9   A1u 
 Orb:    10   A1u 
 Orb:    11  E1uy 
 Orb:    12  E1ux 
 Orb:    13   A1g 
 Orb:    14  E1gx 
 Orb:    15  E1gy 
 Orb:    16   A1g 
 Orb:    17   A1u 
 Orb:    18  E1uy 
 Orb:    19  E1ux 
 Orb:    20  E2gx 
 Orb:    21  E2gy 
 Orb:    22   A1g 
 Orb:    23  E2uy 
 Orb:    24  E2ux 
 Orb:    25  E1gx 
 Orb:    26  E1gy 
 Orb:    27   A1u 
 Orb:    28   A1g 
 Orb:    29  E1uy 
 Orb:    30  E1ux 
 Orb:    31  E1gx 
 Orb:    32  E1gy 
 Orb:    33   A1u 
 Orb:    34  E1uy 
 Orb:    35  E1ux 
 Orb:    36   A1g 
 Orb:    37  E2gx 
 Orb:    38  E2gy 
 Orb:    39  E3uy 
 Orb:    40  E3ux 
 Orb:    41  E3gx 
 Orb:    42  E3gy 
 Orb:    43  E2uy 
 Orb:    44  E2ux 
 Orb:    45  E1gx 
 Orb:    46  E1gy 
 Orb:    47   A1u 
 Orb:    48  E1uy 
 Orb:    49  E1ux 
 Orb:    50  E2gx 
 Orb:    51  E2gy 
 Orb:    52 

In [106]:
init_fspace = [(1,1), (4,4), (1,1), (1,1)]

In [40]:
pyscf.scf.hf_symm.analyze(mf)

**** SCF Summaries ****
Total Energy =                        -108.357518741411909
Nuclear Repulsion Energy =              12.964841667540002
One-electron Energy =                 -172.559523712385669
Two-electron Energy =                   51.237163303433761
TODO: total wave-function symmetry for Dooh
occupancy for each irrep:    A1g E1gx E1gy  A1u E1uy E1ux E2gx E2gy E3gx E3gy E2uy E2ux E3uy E3ux
                               3    0    0    2    1    1    0    0    0    0    0    0    0    0
**** MO energy ****
MO #1 (A1g #1), energy= -15.8274321718833 occ= 2
MO #2 (A1u #1), energy= -15.8271416368937 occ= 2
MO #3 (A1g #2), energy= -1.10438349265382 occ= 2
MO #4 (A1u #2), energy= -0.974783828382259 occ= 2
MO #5 (A1g #3), energy= -0.488182132232768 occ= 2
MO #6 (E1uy #1), energy= -0.410339211636376 occ= 2
MO #7 (E1ux #1), energy= -0.410339211636374 occ= 2
MO #8 (E1gx #1), energy= -0.0933581550076799 occ= 0
MO #9 (E1gy #1), energy= -0.0933581550076848 occ= 0
MO #10 (A1u #3), energy= 0.

((array([1.99999401e+00, 1.95483606e+00, 9.66696475e-04, 3.15288541e-05,
         1.03986022e+00, 9.85108347e-01, 9.85108347e-01, 2.57203718e-03,
         1.29678002e-02, 1.29678002e-02, 5.25489523e-05, 3.13092040e-04,
         3.13092040e-04, 1.38262515e-03, 6.98000988e-34, 3.95938997e-04,
         1.38262515e-03, 1.18781699e-03, 2.00255534e-04, 5.26682981e-34,
         1.83278868e-05, 2.00255534e-04, 5.49836603e-05, 2.61374661e-05,
         9.46434742e-34, 1.74249774e-06, 1.04549864e-05, 1.11855003e-05,
         1.74249774e-05, 1.86425005e-05, 1.99999401e+00, 1.95483606e+00,
         9.66696475e-04, 3.15288541e-05, 1.03986022e+00, 9.85108347e-01,
         9.85108347e-01, 2.57203718e-03, 1.29678002e-02, 1.29678002e-02,
         5.25489523e-05, 3.13092040e-04, 3.13092040e-04, 1.38262515e-03,
         3.59255895e-34, 3.95938997e-04, 1.38262515e-03, 1.18781699e-03,
         2.00255534e-04, 5.72386899e-34, 1.83278868e-05, 2.00255534e-04,
         5.49836603e-05, 2.61374661e-05, 5.87636678

In [107]:
print(clusters)
print(init_fspace)

[[0, 1], [2, 3, 4, 9, 10, 13, 16, 17, 22, 27, 28, 33, 36, 47, 54, 57, 58, 59], [6, 7, 12, 14, 19, 25, 30, 31, 35, 45, 49, 55], [5, 8, 11, 15, 18, 26, 29, 32, 34, 46, 48, 56], [20, 24, 37, 44, 50, 53], [21, 23, 38, 43, 51, 52], [40, 41], [39, 42]]
[(1, 1), (4, 4), (1, 1), (1, 1)]


# Make Integrals

In [110]:
h0 = pyscf.gto.mole.energy_nuc(mol)
h1 = pyscf.scf.hf.get_hcore(mol)
h1 = C_ordered.T @ h1 @ C_ordered

if False:
    d1_embed = 2 * Cdoc @ Cdoc.T

    h  = pyscf.scf.hf.get_hcore(mol)
    j, k = pyscf.scf.hf.get_jk(mol, d1_embed, hermi=1)
    h0 += np.trace(d1_embed @ ( h + .5*j - .25*k))

    h = C_sorted_2.T @ h @ C_sorted_2
    j = C_sorted_2.T @ j @ C_sorted_2;
    k = C_sorted_2.T @ k @ C_sorted_2;

    h1 = h + j - .5*k;


nact = h1.shape[0]

h2 = pyscf.ao2mo.kernel(mol, C_ordered, aosym="s4", compact=False)
h2.shape = (nact, nact, nact, nact)

S = mol.intor("int1e_ovlp_sph")


In [111]:
np.save("integrals_h0", h0)
np.save("integrals_h1", h1)
np.save("integrals_h2", h2)
np.save("mo_coeffs", C_ordered)
np.save("overlap_mat", S)